In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as its
import function_pontius as fpt
import heatmap_cm as hmcm
from pathlib import Path
import openpyxl
import xlsxwriter

# One CM that combines quantity disagreement for all maps

In [2]:
data = []
row_name = []

def heatmap_qd(map1, map2):
    """This function takes in two map names and returns the quantity disagreement for each land-use category.
    The returned values show for each class the percentage (relative to the whole map) of cells that map 1 
    predicted differently than map 2 for that class, with negative values meaning that map 2 predicted more of that category.
    """
    # Load in the maps and set up variables to process the maps
    map1_dir = 'E:/thesis_data/asc_2k/' + map1 + '.asc'
    map2_dir = 'E:/thesis_data/asc_2k/' + map2 + '.asc'    
    m1 = np.loadtxt(map1_dir, skiprows=6)
    m2 = np.loadtxt(map2_dir, skiprows=6)
    cm = fpt.conf_mat(m1, m2)
    # Set up dataframe and adjust for land-use categories that are removed from the df
    rows_cols = ['class' + str(i + 1) for i in range(28)]
    rows_cols.remove('class24')
    rows_cols.remove('class28')
    df = pd.DataFrame(data=cm, index=rows_cols, columns=rows_cols)
    # Create the pontius class and generate the cm matrix with additional info
    df_pont = hmcm.pontiPy(df)
    df_matrix = df_pont.matrix()
    df_matrix['RowDisagreement'] = pd.to_numeric(df_matrix['RowDisagreement'])
    # Assign dummy value to cell to prevent error warning
    df_matrix.loc['ColumnDisagreement', 'RowDisagreement'] = 0
    #Store disagreement
    rowdata = df_matrix.loc[:'class27', 'RowDisagreement']
    coldata = df_matrix.loc['ColumnDisagreement', :'class27']
    qd = rowdata - coldata
    # Format the data to filter out low data and round interesting outcomes
    for i, val in enumerate(qd):
        if val < 0.0001 and val > -0.0001 :
            qd[i] = 0
        else:
            qd[i] = qd[i].round(4)   
    # Create label for new df
    n1 = map1[3:]
    n2 = map2[3:]
    row_id = 'Map ' + n1 + ' & ' + n2
    return qd, row_id
    
def heatmap_to_excel(hm, metric):
    excel_dir = 'E:/thesis_images/representative_maps/excels/heatmap_qd.xlsx'

    writer = pd.ExcelWriter(excel_dir, engine = 'openpyxl')
    book = openpyxl.load_workbook(excel_dir)
    writer.book = book
    # Write each metric to their own sheet
    sheet_id = metric
    hm.to_excel(writer, sheet_name=sheet_id)

    writer.save()
    writer.close()

In [37]:
# Create CM showing quantity disagreement for kappa representative maps
# Overal difference has same maps as kappa so no point 
kappa_data = []
kappa_row_name = []
kappa_maps = ['map359', 'map456', 'map541', 'map695', 'map1148', 'map1760', 'map1911']
iterator = its.permutations(kappa_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    kappa_data.append(dat)
    kappa_row_name.append(row)

df = pd.DataFrame(data=kappa_data, index=kappa_row_name)
df = df.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df = df.loc[:, (df != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
kappa_heatmap = df.style.background_gradient(cmap='coolwarm', axis=None)

In [182]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(25, 25))
hm_kappa = sns.heatmap(df, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .4})
_ = hm_kappa.set_xticklabels(hm_kappa.get_xticklabels(), rotation=60)
_ = hm_kappa.set_yticklabels(hm_kappa.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Kappa', fontsize=20)
cbar = hm_kappa.collections[0].colorbar
cbar.set_label(label='Percentage Difference In Land-Use Category Assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/kappa.png', format='png', bbox_inches="tight")
plt.close()

In [38]:
# Create CM showing quantity disagreement for oa representative maps
oa_data = []
oa_row_name = []
oa_maps = ['map10', 'map359', 'map456', 'map541', 'map695', 'map696', 'map1911']
iterator = its.permutations(oa_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    oa_data.append(dat)
    oa_row_name.append(row)

df2 = pd.DataFrame(data=oa_data, index=oa_row_name)
df2 = df2.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df2 = df2.loc[:, (df2 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
oa_heatmap = df2.style.background_gradient(cmap='coolwarm', axis=None)

In [161]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(25, 25))
hm_oa = sns.heatmap(df2, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .3})
_ = hm_oa.set_xticklabels(hm_oa.get_xticklabels(), rotation=50)
_ = hm_oa.set_yticklabels(hm_oa.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Overall Accuracy', fontsize=20)
cbar = hm_oa.collections[0].colorbar
cbar.set_label(label='Percentage Difference In Land-Use Category Assignment', fontsize=16)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/oa.png', format='png', bbox_inches="tight")
plt.close()

In [39]:
# Create CM showing quantity disagreement for shannon's representative maps
shan_data = []
shan_row_name = []
shan_maps = ['map61', 'map1248', 'map1454', 'map1457']
iterator = its.permutations(shan_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    shan_data.append(dat)
    shan_row_name.append(row)

df3 = pd.DataFrame(data=shan_data, index=shan_row_name)
df3 = df3.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df3 = df3.loc[:, (df3 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
shan_heatmap = df3.style.background_gradient(cmap='coolwarm', axis=None)

In [171]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(17, 12))
hm_shan = sns.heatmap(df3, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .5})
_ = hm_shan.set_xticklabels(hm_shan.get_xticklabels(), rotation=50)
_ = hm_shan.set_yticklabels(hm_shan.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Shannon\'s Diversity Index', fontsize=20)
cbar = hm_shan.collections[0].colorbar
cbar.set_label(label='Percentage Difference In Land-Use Category Assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/shan.png', format='png', bbox_inches="tight")
plt.close()

In [40]:
# Create CM showing quantity disagreement for simpson's representative maps
simp_data = []
simp_row_name = []
simp_maps = ['map65', 'map857', 'map1448', 'map1784', 'map1830']
iterator = its.permutations(simp_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    simp_data.append(dat)
    simp_row_name.append(row)
    
df4 = pd.DataFrame(data=simp_data, index=simp_row_name)
df4 = df4.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df4 = df4.loc[:, (df4 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
simp_heatmap = df4.style.background_gradient(cmap='coolwarm', axis=None)

In [168]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(15, 15))
hm_simp = sns.heatmap(df4, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .5})
_ = hm_simp.set_xticklabels(hm_simp.get_xticklabels(), rotation=50)
_ = hm_simp.set_yticklabels(hm_simp.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Simpson\'s Diversity Index', fontsize=20)
cbar = hm_simp.collections[0].colorbar
cbar.set_label(label='Percentage Difference In Land-Use Category Assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/simp.png', format='png', bbox_inches="tight")
plt.close()

In [41]:
# Create CM showing quantity disagreement for oad representative maps
oad_data = []
oad_row_name = []
oad_maps = ['map263', 'map453', 'map678', 'map1760', 'map1852', 'map1918']
iterator = its.permutations(oad_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    oad_data.append(dat)
    oad_row_name.append(row)
    
df5 = pd.DataFrame(data=oad_data, index=oad_row_name)
df5 = df5.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df5 = df5.loc[:, (df5 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
oad_heatmap = df5.style.background_gradient(cmap='coolwarm', axis=None)

In [176]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(18, 18))
hm_oad = sns.heatmap(df5, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .3})
_ = hm_oad.set_xticklabels(hm_oad.get_xticklabels(), rotation=60)
_ = hm_oad.set_yticklabels(hm_oad.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Overall Allocation Difference', fontsize=20, pad=1)
cbar = hm_oad.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/oad.png', format='png', bbox_inches="tight")
plt.close()

In [42]:
# Create CM showing quantity disagreement for oqd representative maps
oqd_data = []
oqd_row_name = []
oqd_maps = ['map1057', 'map1472', 'map1833', 'map1978']
iterator = its.permutations(oqd_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    oqd_data.append(dat)
    oqd_row_name.append(row)
    
df6 = pd.DataFrame(data=oqd_data, index=oqd_row_name)
df6 = df6.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df6 = df6.loc[:, (df6 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
oqd_heatmap = df6.style.background_gradient(cmap='coolwarm', axis=None)

In [189]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_oqd = sns.heatmap(df6, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .4})
_ = hm_oqd.set_xticklabels(hm_oqd.get_xticklabels(), rotation=65)
_ = hm_oqd.set_yticklabels(hm_oqd.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Overall Quantity Difference', fontsize=20, pad=1)
cbar = hm_oqd.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/oqd.png', format='png', bbox_inches="tight")
plt.close()

In [43]:
# Create CM showing quantity disagreement for tca4 representative maps
tca4_data = []
tca4_row_name = []
tca4_maps = ['map217', 'map332', 'map604']
iterator = its.permutations(tca4_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    tca4_data.append(dat)
    tca4_row_name.append(row)
    
df7 = pd.DataFrame(data=tca4_data, index=tca4_row_name)
df7 = df7.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df7 = df7.loc[:, (df7 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
tca4_heatmap = df7.style.background_gradient(cmap='coolwarm', axis=None)

In [196]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_tca4 = sns.heatmap(df7, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .3})
_ = hm_tca4.set_xticklabels(hm_tca4.get_xticklabels(), rotation=50)
_ = hm_tca4.set_yticklabels(hm_tca4.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Total Class Area [Residential]', fontsize=20, pad=1)
cbar = hm_tca4.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/tca4.png', format='png', bbox_inches="tight")
plt.close()

In [44]:
# Create CM showing quantity disagreement for tca4 representative maps
pland4_data = []
pland4_row_name = []
pland4_maps = ['map264', 'map1422', 'map1877']
iterator = its.permutations(pland4_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    pland4_data.append(dat)
    pland4_row_name.append(row)
    
df8 = pd.DataFrame(data=pland4_data, index=pland4_row_name)
df8 = df8.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df8 = df8.loc[:, (df8 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
pland4_heatmap = df8.style.background_gradient(cmap='coolwarm', axis=None)

In [198]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_pland4 = sns.heatmap(df8, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .3})
_ = hm_pland4.set_xticklabels(hm_pland4.get_xticklabels(), rotation=50)
_ = hm_pland4.set_yticklabels(hm_pland4.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n Percentage Landscape [Residential]', fontsize=20, pad=1)
cbar = hm_pland4.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/pland4.png', format='png', bbox_inches="tight")
plt.close()

In [45]:
# Create CM showing quantity disagreement for tca4 representative maps
tca14_data = []
tca14_row_name = []
tca14_maps = ['map246', 'map296', 'map325', 'map825', 'map1612']
iterator = its.permutations(tca14_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    tca14_data.append(dat)
    tca14_row_name.append(row)
    
df9 = pd.DataFrame(data=tca14_data, index=tca14_row_name)
df9 = df9.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df9 = df9.loc[:, (df9 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
tca14_heatmap = df9.style.background_gradient(cmap='coolwarm', axis=None)

In [203]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_tca14 = sns.heatmap(df9, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .5})
_ = hm_tca14.set_xticklabels(hm_tca14.get_xticklabels(), rotation=65)
_ = hm_tca14.set_yticklabels(hm_tca14.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Total Class Area [Corn]', fontsize=20, pad=1)
cbar = hm_tca14.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/tca14.png', format='png', bbox_inches="tight")
plt.close()

In [46]:
# Create CM showing quantity disagreement for tca4 representative maps
pland14_data = []
pland14_row_name = []
pland14_maps = ['map246', 'map325', 'map564', 'map704', 'map843']
iterator = its.permutations(pland14_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    pland14_data.append(dat)
    pland14_row_name.append(row)
    
df10 = pd.DataFrame(data=pland14_data, index=pland14_row_name)
df10 = df10.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df10 = df10.loc[:, (df10 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
pland14_heatmap = df10.style.background_gradient(cmap='coolwarm', axis=None)

In [207]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_pland14 = sns.heatmap(df10, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .5})
_ = hm_pland14.set_xticklabels(hm_pland14.get_xticklabels(), rotation=65)
_ = hm_pland14.set_yticklabels(hm_pland14.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Percentage Landscape [Corn]', fontsize=20, pad=1)
cbar = hm_pland14.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)
plt.savefig('E:/thesis_images/representative_maps/cm_figures/pland14.png', format='png', bbox_inches="tight")
plt.close()

In [4]:
# Create CM showing quantity disagreement for tca4 representative maps
# TCA22 and PLAND 22 have the same representative maps so only TCA will be done for both of them
tca22_data = []
tca22_row_name = []
tca22_maps = ['map0', 'map8', 'map35']
iterator = its.permutations(tca22_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    tca22_data.append(dat)
    tca22_row_name.append(row)
    
df11 = pd.DataFrame(data=tca22_data, index=tca22_row_name)
df11 = df11.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df11 = df11.loc[:, (df11 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
tca22_heatmap = df11.style.background_gradient(cmap='coolwarm', axis=None)

In [212]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_tca22 = sns.heatmap(df11, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .3})
_ = hm_tca22.set_xticklabels(hm_tca22.get_xticklabels(), rotation=65)
_ = hm_tca22.set_yticklabels(hm_tca22.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Total Class Area [Nature]', fontsize=20, pad=1)
cbar = hm_tca22.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)

plt.savefig('E:/thesis_images/representative_maps/cm_figures/tca22.png', format='png', bbox_inches="tight")
plt.close()

In [35]:
# Create CM showing quantity disagreement for qd22 representative maps
qd22_data = []
qd22_row_name = []
qd22_maps = ['map0', 'map1', 'map8']
iterator = its.permutations(qd22_maps, 2)

for dat, row in its.starmap(heatmap_qd, iterator):
    qd22_data.append(dat)
    qd22_row_name.append(row)
    
df12 = pd.DataFrame(data=qd22_data, index=qd22_row_name)
df12 = df12.rename(columns={'class1':'Railway', 'class2':'Road', 'class3':'Airport', 'class4':'Residential', 'class5':'Industrial', 'class6':'Facillities', 'class7':'Ports' \
        ,'class8':'Construction Area', 'class9':'Semi hardened area', 'class10':'Recreational (daily)', 'class11':'Recreational (residence)' \
       , 'class12':'Grass (residential area)', 'class13':'Greenhouse Horticulture', 'class14':'Corn', 'class15':'Potatoes', 'class16':'Beets' \
        , 'class17':'Grains', 'class18':'Openground Vegatables', 'class19':'Flower bulbs', 'class20':'Pasturage' \
       , 'class21':'Orchard', 'class22':'Arboriculture', 'class23':'Nature', 'class25':'Large Fresh Water', 'class26':'Rivers', 'class27':'Misc Waters'}) 
df12 = df12.loc[:, (df12 != 0).any(axis=0)]
# Apply heatmap to cm using pandas styler
qd22_heatmap = df12.style.background_gradient(cmap='coolwarm', axis=None)

In [217]:
# Create seaborn heatmap and save to disk
plt.figure(figsize=(12, 12))
hm_qd22 = sns.heatmap(df12, cmap='coolwarm', fmt=".4f", annot=True, square=True, annot_kws={"size": 8} \
                , cbar_kws={'label': 'Percentage difference in land-use category assignment','pad': 0.015, 'shrink': .25})
_ = hm_qd22.set_xticklabels(hm_qd22.get_xticklabels(), rotation=60)
_ = hm_qd22.set_yticklabels(hm_qd22.get_yticklabels(), rotation=0)
plt.ylabel('Representative Map Pair', fontsize=16)
plt.xlabel('Land-Use Category', fontsize=16) 
plt.title('Representative Map Comparisons \n For Quantity Difference [Nature]', fontsize=20, pad=1)
cbar = hm_qd22.collections[0].colorbar
cbar.set_label(label='Percentage difference in land-use category assignment', fontsize=12)
plt.savefig('E:/thesis_images/representative_maps/cm_figures/qd22.png', format='png', bbox_inches="tight")
plt.close()

In [21]:
# Store all heatmaps in an excel
heatmap_to_excel(kappa_heatmap, 'kappa')
heatmap_to_excel(oa_heatmap, 'oa')
heatmap_to_excel(shan_heatmap, 'shan')
heatmap_to_excel(simp_heatmap, 'simp')
heatmap_to_excel(oad_heatmap, 'oad')
heatmap_to_excel(oqd_heatmap, 'oqd')
heatmap_to_excel(tca4_heatmap, 'tca4')
heatmap_to_excel(pland4_heatmap, 'pland4')
heatmap_to_excel(tca14_heatmap, 'tca14')
heatmap_to_excel(pland14_heatmap, 'pland14')
heatmap_to_excel(tca22_heatmap, 'tca22')
heatmap_to_excel(qd22_heatmap, 'qd22')